[실습문제] 와인 퀄리티 예측
#115


https://www.notion.so/shqkel/2b19cb46e5e281029ce8f76084793ffb?source=copy_link

<img width="1800" height="400" alt="Image" src="https://github.com/user-attachments/assets/3f9a4afe-9f81-41db-a062-e061b674ceff" />

와인 품질 데이터셋을 기반으로 0 ~ 10사이의 와인품질을 분류 예측하는 모델을 구현하세요.
(실제로는 3 ~ 8사이의 클래스를 예측하면 됩니다.)

In [2]:
import pandas as pd

# 와인 퀄리티 분류
# - 0 ~ 10까지 타겟클래스 예측 (실제로는 3 ~ 8사이를 예측)
# - 정확도/정밀도/재현율/F1점수까지 확인 (classfication_report)
wine_quality_df = pd.read_csv('data/winequality-red.csv')
print(wine_quality_df['quality'].value_counts().sort_index())
wine_quality_df.head()

quality
3     10
4     53
5    681
6    638
7    199
8     18
Name: count, dtype: int64


,fixed acidity,volatile acidity,citric acid,residual sugar,chlorides,free sulfur dioxide,total sulfur dioxide,density,pH,sulphates,alcohol,quality
0,7.4,0.70,0.00,1.9,0.076,11.0,34.0,0.9978,3.51,0.56,9.4,5
1,7.8,0.88,0.00,2.6,0.098,25.0,67.0,0.9968,3.20,0.68,9.8,5
2,7.8,0.76,0.04,2.3,0.092,15.0,54.0,0.9970,3.26,0.65,9.8,5
3,11.2,0.28,0.56,1.9,0.075,17.0,60.0,0.9980,3.16,0.58,9.8,6
4,7.4,0.70,0.00,1.9,0.076,11.0,34.0,0.9978,3.51,0.56,9.4,5


In [5]:
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

X = wine_quality_df.drop('quality', axis=1).to_numpy()
y = wine_quality_df['quality'].to_numpy()

X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=0, stratify=y)

scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

In [6]:
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import classification_report

model = LogisticRegression()
model.fit(X_train, y_train)

print(f'학습 Accuracy: {model.score(X_train, y_train)}')
print(f'평가 Accuracy: {model.score(X_test, y_test)}')

print(classification_report(y_test, model.predict(X_test), zero_division=0))

학습 Accuracy: 0.6030025020850709
평가 Accuracy: 0.6025
              precision    recall  f1-score   support

           3       0.50      0.50      0.50         2
           4       1.00      0.08      0.14        13
           5       0.67      0.73      0.70       170
           6       0.55      0.61      0.58       160
           7       0.51      0.36      0.42        50
           8       0.00      0.00      0.00         5

    accuracy                           0.60       400
   macro avg       0.54      0.38      0.39       400
weighted avg       0.60      0.60      0.59       400



In [7]:
model = LogisticRegression(class_weight='balanced') # 개수가 적은 클래스에 더많은 가중치 부여해서 좀더 쉽게 예측하도록함
model.fit(X_train, y_train)

print(f'학습 Accuracy: {model.score(X_train, y_train)}')
print(f'평가 Accuracy: {model.score(X_test, y_test)}')

print(classification_report(y_test, model.predict(X_test)))

학습 Accuracy: 0.42952460383653046
평가 Accuracy: 0.41
              precision    recall  f1-score   support

           3       0.05      0.50      0.09         2
           4       0.04      0.23      0.07        13
           5       0.70      0.52      0.60       170
           6       0.62      0.28      0.39       160
           7       0.31      0.46      0.37        50
           8       0.10      0.80      0.18         5

    accuracy                           0.41       400
   macro avg       0.30      0.46      0.28       400
weighted avg       0.59      0.41      0.46       400



In [8]:
# 타겟클래스 단순화 해서 예측하기
print(wine_quality_df['quality'].value_counts().sort_index())

# 변환
def map_quality_to_category(q):
    if q <= 4:
        return 0
    elif q <= 6:
        return 1
    else:
        return 2

wine_quality_df['quality_simple'] = wine_quality_df['quality'].apply(map_quality_to_category)
print(wine_quality_df['quality_simple'].value_counts().sort_index())

# 입력/라벨 데이터분리
X = wine_quality_df.drop(['quality', 'quality_simple'], axis=1).to_numpy()
y = wine_quality_df['quality_simple'].to_numpy()

X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=0, stratify=y)

scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

quality
3     10
4     53
5    681
6    638
7    199
8     18
Name: count, dtype: int64
quality_simple
0      63
1    1319
2     217
Name: count, dtype: int64


In [9]:
model = LogisticRegression(class_weight='balanced') # 개수가 적은 클래스에 더많은 가중치 부여해서 좀더 쉽게 예측하도록함
model.fit(X_train, y_train)

print(f'학습 Accuracy: {model.score(X_train, y_train)}')
print(f'평가 Accuracy: {model.score(X_test, y_test)}')

print(classification_report(y_test, model.predict(X_test)))

학습 Accuracy: 0.6013344453711427
평가 Accuracy: 0.5475
              precision    recall  f1-score   support

           0       0.12      0.81      0.21        16
           1       0.93      0.51      0.66       330
           2       0.33      0.69      0.45        54

    accuracy                           0.55       400
   macro avg       0.46      0.67      0.44       400
weighted avg       0.82      0.55      0.61       400

